# Funcionalidades

In [11]:
from ultralytics import YOLO

In [4]:
!pip install ultralytics


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached ultralytics-8.3.78-py3-none-any.whl (921 kB)
  Using cached numpy-2.1.1-cp311-cp311-win_amd64.whl (12.9 MB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)
  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl (204.2 MB)
  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl (1.6 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
  Using cached filelock-3.17.0-py3-none-any.whl (16 kB)
  Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
  Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl (184 kB)
  Using cached sympy-1.13.

In [2]:
# Cargar modelo YOLO11s fine-tuned


# Cargar modelo YOLO
model = YOLO(r'modelos\best.pt')

# Imprimir las clases disponibles
print("Clases disponibles en el modelo:", model.names)

Clases disponibles en el modelo: {0: 'cliente', 1: 'empleado'}


##  Cantidad de clientes por zona

In [72]:
# Funcion para determinar coordenadas de objetos en videos
import cv2

# Función de callback para obtener coordenadas al hacer clic
def obtener_coordenadas(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Solo muestra al hacer clic izquierdo
        print(f'Coordenadas: ({x}, {y})')

# Capturar un frame del video
cap = cv2.VideoCapture('C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    cv2.imshow('Frame', frame)
    cv2.setMouseCallback('Frame', obtener_coordenadas)  # Asigna la función al frame

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No se pudo leer el frame del video.")



Coordenadas: (199, 115)
Coordenadas: (424, 285)


In [12]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

In [94]:
# Función para calcular la distancia entre un punto (px, py) y una línea definida por dos puntos (x1, y1) y (x2, y2)
def distancia_punto_linea(px, py, x1, y1, x2, y2):
    return abs((y2 - y1) * px - (x2 - x1) * py + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

In [93]:
# Definir las coordenadas de la línea del mostrador (inicio y fin)
x1_mostrador, y1_mostrador = 199, 115
x2_mostrador, y2_mostrador = 424, 285

In [ ]:
# Función para contar los clientes cerca de la línea del mostrador
def contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador):
    count = 0
    clientes_detectados = []  # Lista para almacenar las coordenadas de los clientes detectados
    for deteccion in detecciones:
        bbox = deteccion.xywh[0]
        x, y = bbox[0].item(), bbox[1].item()  # Centro en formato xywh
        clase = int(deteccion.cls[0].item())

        if clase == 1:
            distancia = distancia_punto_linea(x, y, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)
            if distancia < 30:  # Umbral para considerar que está en la zona del mostrador
                count += 1
                clientes_detectados.append((x, y))  # Guardar la coordenada de los clientes

    return count, clientes_detectados

In [103]:
# Leer video
import cv2
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4")

# Lista para almacenar los resultados
resultados = []

# Extraer la fecha y hora inicial desde el nombre del archivo
nombre_video = "grabacion_2025-02-21_19-42-20.mp4"
fecha_hora_inicial_str = nombre_video.split('_')[1] + " " + nombre_video.split('_')[2].replace('.mp4', '')
fecha_hora_inicial = datetime.strptime(fecha_hora_inicial_str, "%Y-%m-%d %H-%M-%S")

In [ ]:

# crea un csv con la cantidad de clientes cerca de mostrador detectados

# Procesar cada fotograma
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obtener el número de fotograma y calcular el tiempo en segundos
    frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    fps = video.get(cv2.CAP_PROP_FPS)
    segundos_transcurridos = frame_number / fps

    # Calcular la nueva fecha y hora
    nueva_fecha_hora = fecha_hora_inicial + timedelta(seconds=segundos_transcurridos)
    fecha_hora_exacta = nueva_fecha_hora.strftime("%Y-%m-%d %H:%M:%S")

    # Realizar la detección de objetos (clientes y empleados)
    resultados_deteccion = model(frame)

    # Extraer las detecciones
    detecciones = resultados_deteccion[0].boxes if resultados_deteccion else []

    # Contar clientes cerca del mostrador y obtener sus coordenadas
    clientes_mostrador, clientes_detectados = contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

    # Guardar la información del primer cliente detectado (si hay alguno)
    if clientes_detectados:
        x_cliente, y_cliente = clientes_detectados[0]  # Tomar la primera detección

        # Guardar la detección en la lista
        resultados.append([fecha_hora_exacta, clientes_mostrador, x_cliente, y_cliente])

# Guardar los resultados en un CSV
df = pd.DataFrame(resultados, columns=['Fecha Hora y Segundo', 'Clientes Cerca del Mostrador', 'X Cliente', 'Y Cliente'])
df.to_csv('resultados_deteccion_mostrador_con_ubicaciones.csv', index=False)

# Liberar el video y cerrar ventanas
video.release()
cv2.destroyAllWindows()

0: 384x640 4 clientes, 2 mostradors, 350.2ms
Speed: 8.9ms preprocess, 350.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 303.3ms
Speed: 2.1ms preprocess, 303.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 325.0ms
Speed: 2.2ms preprocess, 325.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 323.9ms
Speed: 2.6ms preprocess, 323.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 283.2ms
Speed: 3.4ms preprocess, 283.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 343.0ms
Speed: 2.4ms preprocess, 343.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 477.1ms
Speed: 2.6ms preprocess, 477.1ms inference, 1.9ms postprocess per image a

In [ ]:

# crea un csv con la cantidad de clientes detectados en total y los que estan cerca de mostrador
resultados_clientes_totales = []

# Procesar cada fotograma
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obtener el número de fotograma y calcular el tiempo en segundos
    frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    fps = video.get(cv2.CAP_PROP_FPS)
    segundos_transcurridos = frame_number / fps

    # Calcular la nueva fecha y hora
    nueva_fecha_hora = fecha_hora_inicial + timedelta(seconds=segundos_transcurridos)
    fecha_hora_exacta = nueva_fecha_hora.strftime("%Y-%m-%d %H:%M:%S")

    # Realizar la detección de objetos (clientes y empleados)
    resultados_deteccion = model(frame)

    # Extraer las detecciones
    detecciones = resultados_deteccion[0].boxes if resultados_deteccion else []

    # Contar clientes cerca del mostrador y obtener sus coordenadas
    clientes_mostrador, clientes_detectados = contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

    # Contar todos los clientes detectados en el fotograma
    clientes_totales = sum(1 for deteccion in detecciones if int(deteccion.cls[0].item()) == 1)  # Suponiendo que 1 es la clase "cliente"

    # Guardar la información del primer cliente detectado (si hay alguno)
    if clientes_detectados:
        x_cliente, y_cliente = clientes_detectados[0]  # Tomar la primera detección

        # Guardar la detección en la lista de resultados
        resultados_clientes_totales.append([fecha_hora_exacta, clientes_mostrador, clientes_totales, x_cliente, y_cliente])

# Guardar los resultados de detección con clientes totales en un CSV
df_clientes_totales = pd.DataFrame(resultados_clientes_totales, columns=['Fecha Hora y Segundo', 'Clientes Cerca del Mostrador', 'Clientes Totales', 'X Cliente', 'Y Cliente'])
df_clientes_totales.to_csv('resultados_clientes_totales.csv', index=False)

# Liberar el video y cerrar ventanas
video.release()
cv2.destroyAllWindows()


0: 384x640 4 clientes, 2 mostradors, 351.3ms
Speed: 13.4ms preprocess, 351.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 311.6ms
Speed: 2.6ms preprocess, 311.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 2 mostradors, 377.6ms
Speed: 3.4ms preprocess, 377.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 390.8ms
Speed: 2.5ms preprocess, 390.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 373.2ms
Speed: 2.6ms preprocess, 373.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 444.7ms
Speed: 2.3ms preprocess, 444.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 307.7ms
Speed: 2.4ms preprocess, 307.7ms inference, 1.5ms postprocess per image

In [35]:
import cv2

# Cargar el video
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4")

# Saltar los primeros 4 frames para llegar al quinto
frame_count = 0
while frame_count < 100:
    ret, frame = video.read()
    frame_count += 1
    if not ret:
        print("No se pudo leer el frame 5.")
        video.release()
        exit()

# Definir la coordenada del objeto detectado (según el CSV)
x, y = 436.643310546875,313.2765808105469 # Centro de la caja

# Dibujar el punto del objeto detectado
cv2.circle(frame, (int(x), int(y)), 5, (255, 0, 0), -1)  # Azul

# Dibujar la línea del mostrador
cv2.line(frame, (125, 312), (147, 358), (0, 0, 255), 2)  # Rojo

# Mostrar el frame
cv2.imshow("Detección", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar el video
video.release()


## Tiempo de permanencia clientes

### DeepSort

In [23]:
import cv2
import torch
import pandas as pd
import numpy as np 
from datetime import datetime, timedelta


In [ ]:
# pip install deep_sort_realtime

In [13]:
from deep_sort_realtime.deep_sort.detection import Detection
from deep_sort_realtime.deep_sort.tracker import Tracker
from deep_sort_realtime.deep_sort.nn_matching import NearestNeighborDistanceMetric
from deep_sort_realtime.deep_sort.kalman_filter import KalmanFilter


In [14]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [36]:
# Diccionario para almacenar tiempos de entrada y salida
clientes = {}
registros = []

In [104]:
import cv2

def reducir_fps(video_path, nuevo_fps, output_path):
    """
    Reduce los FPS de un video y guarda el resultado
    
    Parámetros:
    video_path (str): Ruta del video de entrada
    nuevo_fps (int/float): Nuevo número de FPS deseado (debe ser menor que el original)
    output_path (str): Ruta donde se guardará el video resultante
    
    Retorna:
    None
    """
    
    # Abrir el video de entrada
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("No se pudo abrir el video de entrada")
    
    # Obtener propiedades del video original
    orig_fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Validar nuevo FPS
    if nuevo_fps <= 0 or nuevo_fps >= orig_fps:
        cap.release()
        raise ValueError("El nuevo FPS debe ser menor que el original y mayor que 0")
    
    # Configurar el escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, nuevo_fps, (width, height))
    
    if not out.isOpened():
        cap.release()
        raise ValueError("No se pudo inicializar el escritor de video")
    
    # Calcular intervalo de frames
    step = orig_fps / nuevo_fps
    current = 0.0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            current += 1
            if current >= step:
                out.write(frame)
                current -= step
                
    finally:
        # Liberar recursos
        cap.release()
        out.release()

# Ejemplo de uso:
reducir_fps('C:\\Users\\alvar\\Desktop\\Proyecto-UEM\\ProyectoGrupal-G3-UEM\\grabacion_2025-02-20_18-38-13.mp4', 10, 'C:\\Users\\alvar\\Desktop\\Proyecto-UEM\\ProyectoGrupal-G3-UEM\\grabacion_2025-02-20_18-38-14.mp4')

In [111]:
# Configuración inicial
VIDEO_PATH = "C:\\Users\\alvar\\Desktop\\Proyecto-UEM\\ProyectoGrupal-G3-UEM\\grabacion_2025-02-20_18-38-12.mp4"
CSV_PATH = 'tiempos_permanencia.csv'
CONFIDENCE_THRESHOLD = 0.6
MAX_AGE = 150  # Número de frames para mantener un track sin actualizaciones

In [113]:
# Inicializar modelos
model = YOLO('C:\\Users\\alvar\\Desktop\\Proyecto-UEM\\ProyectoGrupal-G3-UEM\\modelos\\best.pt')
# Inicializar DeepSort con parámetros ajustados
tracker = DeepSort(
    max_age=MAX_AGE,         # Tiempo máximo para mantener un track sin actualización
    max_cosine_distance=0.3, # Umbral de distancia para asignar un track a una detección
    max_iou_distance=0.6,    # Umbral de IoU para reasociar tracks
    n_init=5,             # Frames necesarios para confirmar un track (reduce falsos positivos)
)
metric = NearestNeighborDistanceMetric("cosine", matching_threshold=0.2, budget=100)
tracker.metric = metric
tracker.kf = KalmanFilter()



In [114]:
# Estructuras de datos
active_customers = {}
results = []
frame_count = 0

In [115]:
# Procesar video
cap = cv2.VideoCapture(VIDEO_PATH)

# Obtener FPS del video
FRAMES_PER_SECOND = cap.get(cv2.CAP_PROP_FPS)

In [116]:
FRAMES_PER_SECOND

10.0

In [117]:

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    current_time = frame_count / FRAMES_PER_SECOND

    # Detección con YOLO
    yolo_results = model(frame)  # Resultado de YOLO
    boxes = yolo_results[0].boxes

    # Extraer datos
    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls = boxes.cls.cpu().numpy()
    detections = np.column_stack((xyxy, conf, cls))

    # Filtrar clientes (clase 0)
    customer_mask = detections[:, 5] == 0
    customer_detections = detections[customer_mask]

    # Preparar para DeepSort
    converted_detections = []
    for det in customer_detections:
        x1, y1, x2, y2, conf, cls = det[:6]
        if conf >= CONFIDENCE_THRESHOLD:
            w = x2 - x1
            h = y2 - y1
            converted_detections.append(([x1, y1, w, h], conf, cls))

    # Actualizar tracks
    tracks = tracker.update_tracks(converted_detections, frame=frame)
    
    current_ids = set()
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        current_ids.add(track_id)
        
        if track_id not in active_customers:
            active_customers[track_id] = {
                'entry_time': current_time,
                'last_seen': current_time
            }
        else:
            active_customers[track_id]['last_seen'] = current_time

    to_remove = []
    for track_id in active_customers:
        if track_id not in current_ids:
            time_since_last_seen = current_time - active_customers[track_id]['last_seen']
            if time_since_last_seen > MAX_AGE / FRAMES_PER_SECOND:
                entry = active_customers[track_id]['entry_time']
                exit = active_customers[track_id]['last_seen']
                customer_data = {
                    'cliente_id': track_id,
                    'entrada': entry,
                    'salida': exit,
                    'tiempo_permanencia': exit - entry
                }
                results.append(customer_data)
                to_remove.append(track_id)
    
    for track_id in to_remove:
        del active_customers[track_id]

# Procesar clientes restantes
for track_id, data in active_customers.items():
    customer_data = {
        'cliente_id': track_id,
        'entrada': data['entry_time'],
        'salida': data['last_seen'],
        'tiempo_permanencia': data['last_seen'] - data['entry_time']
    }
    results.append(customer_data)

# Depuración
print("Contenido de results antes de filtrar:", results)

# Filtrar solo los diccionarios válidos
valid_results = [item for item in results if isinstance(item, dict) and 'cliente_id' in item]
print("Contenido de results después de filtrar:", valid_results)

# Crear DataFrame
df = pd.DataFrame(valid_results, columns=['cliente_id', 'entrada', 'salida', 'tiempo_permanencia'])

# Guardar en CSV
df.to_csv(CSV_PATH, index=False)

# Liberar recursos
cap.release()
print("CSV generado con éxito en:", CSV_PATH)


0: 384x640 1 cliente, 308.5ms
Speed: 5.5ms preprocess, 308.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 282.6ms
Speed: 3.1ms preprocess, 282.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 297.5ms
Speed: 3.8ms preprocess, 297.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 270.7ms
Speed: 3.8ms preprocess, 270.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 265.0ms
Speed: 3.2ms preprocess, 265.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 311.9ms
Speed: 4.5ms preprocess, 311.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 274.2ms
Speed: 3.0ms preprocess, 274.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cliente, 257.5ms
Speed: 3.0ms preprocess, 257.5ms inference, 1.3ms postprocess per 

In [110]:
# Codigo para ver en video como va funciona el tracker
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    current_time = frame_count / FRAMES_PER_SECOND

    # Detección con YOLO
    yolo_results = model(frame)
    boxes = yolo_results[0].boxes

    # Extraer datos
    xyxy = boxes.xyxy.cpu().numpy()
    conf = boxes.conf.cpu().numpy()
    cls = boxes.cls.cpu().numpy()
    detections = np.column_stack((xyxy, conf, cls))

    # Filtrar clientes (clase 0)
    customer_mask = detections[:, 5] == 0
    customer_detections = detections[customer_mask]

    # Preparar para DeepSort
    converted_detections = []
    for det in customer_detections:
        x1, y1, x2, y2, conf, cls = det[:6]
        if conf >= CONFIDENCE_THRESHOLD:
            w = x2 - x1
            h = y2 - y1
            converted_detections.append(([x1, y1, w, h], conf, cls))

    # Actualizar tracks
    tracks = tracker.update_tracks(converted_detections, frame=frame)
    
    current_ids = set()
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        current_ids.add(track_id)
        
        # Obtener las coordenadas del bounding box
        bbox = track.to_tlbr()  # Top-left-bottom-right
        x1, y1, x2, y2 = map(int, bbox)

        # Dibujar el bounding box y el ID en el fotograma
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        if track_id not in active_customers:
            active_customers[track_id] = {
                'entry_time': current_time,
                'last_seen': current_time
            }
        else:
            active_customers[track_id]['last_seen'] = current_time

    to_remove = []
    for track_id in active_customers:
        if track_id not in current_ids:
            time_since_last_seen = current_time - active_customers[track_id]['last_seen']
            if time_since_last_seen > MAX_AGE / FRAMES_PER_SECOND:
                entry = active_customers[track_id]['entry_time']
                exit = active_customers[track_id]['last_seen']
                customer_data = {
                    'cliente_id': track_id,
                    'entrada': entry,
                    'salida': exit,
                    'tiempo_permanencia': exit - entry
                }
                results.append(customer_data)
                to_remove.append(track_id)
    
    for track_id in to_remove:
        del active_customers[track_id]

    # Mostrar el fotograma con los bounding boxes
    cv2.imshow('Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Presiona 'q' para salir
        break

# Procesar clientes restantes
for track_id, data in active_customers.items():
    customer_data = {
        'cliente_id': track_id,
        'entrada': data['entry_time'],
        'salida': data['last_seen'],
        'tiempo_permanencia': data['last_seen'] - data['entry_time']
    }
    results.append(customer_data)

# Filtrar solo los diccionarios válidos
valid_results = [item for item in results if isinstance(item, dict) and 'cliente_id' in item]

# Crear DataFrame
df = pd.DataFrame(valid_results, columns=['cliente_id', 'entrada', 'salida', 'tiempo_permanencia'])

# Guardar en CSV
df.to_csv(CSV_PATH, index=False)

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
print("CSV generado con éxito en:", CSV_PATH)


0: 384x640 (no detections), 495.9ms
Speed: 55.6ms preprocess, 495.9ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 451.5ms
Speed: 6.8ms preprocess, 451.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 415.6ms
Speed: 8.9ms preprocess, 415.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 378.1ms
Speed: 5.4ms preprocess, 378.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 316.9ms
Speed: 4.4ms preprocess, 316.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 293.0ms
Speed: 4.8ms preprocess, 293.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 292.8ms
Speed: 4.9ms preprocess, 292.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 297.5ms
Speed: 8.5ms pre